# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada-filtrados-500.csv', sep = '|')
df['filtrado_500']=df['filtrado_500'].astype(str)
df.head()

,acordao,areas,filtrado_500
0,297/2016-P,Responsabilidade,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,senac senac senac empresa senac giselle araújo...


In [2]:
df.shape

(10524, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 35000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_500'])

sequences = tokenizer.texts_to_sequences(df['filtrado_500'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 33339 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 500)


In [7]:
x.shape, y.shape

((10524, 500), (10524, 91))

# Treinamento

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Bidirectional
from keras.optimizers import RMSprop
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0314 19:56:49.003989 140349225908032 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0314 19:56:49.039267 140349225908032 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0314 19:56:49.042184 140349225908032 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0314 19:56:49.290944 140349225908032 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 91)                23387     
Total params: 3,797,563
Trainable params: 3,797,563
Non-trainable params: 0
_________________________________________________________________


W0314 19:56:49.902762 140349225908032 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 111s 13ms/step - loss: 2.0637 - categorical_accuracy: 0.4378 - val_loss: 1.6945 - val_categorical_accuracy: 0.5235
Epoch 2/20
8419/8419 [==============================] - 110s 13ms/step - loss: 2.0302 - categorical_accuracy: 0.5244 - val_loss: 1.6946 - val_categorical_accuracy: 0.5183
Epoch 3/20
8419/8419 [==============================] - 109s 13ms/step - loss: 1.4183 - categorical_accuracy: 0.6092 - val_loss: 2.1187 - val_categorical_accuracy: 0.3900
Epoch 4/20
8419/8419 [==============================] - 109s 13ms/step - loss: 1.2641 - categorical_accuracy: 0.6587 - val_loss: 1.8502 - val_categorical_accuracy: 0.5192
Epoch 5/20
8419/8419 [==============================] - 110s 13ms/step - loss: 1.1175 - categorical_accuracy: 0.6990 - val_loss: 1.7230 - val_categorical_accuracy: 0.5325
Epoch 6/20
8419/8419 [==============================] - 110s 13ms/step - loss: 0.9851 - categoric

In [9]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0314 20:33:28.674296 140349225908032 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 91)                23387     
Total params: 3,797,563
Trainable params: 3,797,563
Non-trainable params: 0
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 130s 15ms/step - loss: 2.0410 - categorical_accuracy: 0.4391 - val_loss: 1.8161 - val_categorical_accuracy: 0.4831
Epoch 2/20
8419/8419 [==============================] - 129s 15ms/step - loss: 1.5350 - categorical_accuracy: 0.5783 - val_loss: 1.6935 - val_categorical_accuracy: 0.4960
Epoch 

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_3 (Dense)              (None, 91)                23387     
Total params: 3,797,563
Trainable params: 3,797,563
Non-trainable params: 0
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 132s 16ms/step - loss: 2.0575 - categorical_accuracy: 0.4320 - val_loss: 1.7899 - val_categorical_accuracy: 0.3691
Epoch 2/20
8419/8419 [==============================] - 130s 15ms/step - loss: 1.5630 - categorical_accuracy: 0.5762 - val_loss: 1.6774 - val_categorical_accuracy: 0.4613
Epoch 